In [1]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, LSTM, Dense, TimeDistributed, Flatten
import pandas as pd
import cv2
import numpy as np
import logging

logging.basicConfig(level=logging.INFO)

2023-06-26 18:43:18.888240: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-26 18:43:19.172157: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-26 18:43:19.173812: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-26 18:43:20.122607: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
def load_data():
    try:
        with open('gbn-fonts.tsv', 'r') as file:
            data = pd.read_csv(file, sep='\t')
            
        data.dropna()
            
        data.drop(data[(data['font_family'] != 'Fraktur')].index, axis=0, inplace=True)
        
        return data
        
#         print(data[2511])

#         image_paths = "dataset/Fraktur/" + data['segment_id'] + ".png"
#         labels = data['text_equiv']

# #         images = []
# #         for path in image_paths:
# #             try:
# #                 image = cv2.imread(path, 0)
# #                 if image is None:
# #                     logging.warning(f"Failed to read image: {path}")
# #                     continue

# #                 # Perform image dilation (expanding) and erosion
# # #                 kernel = np.ones((3, 3), np.uint8)
# # #                 dilated_image = cv2.dilate(image, kernel, iterations=1)
# # #                 eroded_image = cv2.erode(image, kernel, iterations=1)

# # #                 # Resize the images if needed
# # #                 dilated_image = cv2.resize(dilated_image, (224, 224))
# # #                 eroded_image = cv2.resize(eroded_image, (224, 224))

# #                 images.append(image)
# #             except Exception as e:
# #                 logging.warning(f"Error processing image: {path}, {str(e)}")

# #         images = np.array(images, dtype="float32")
#         return image_paths, labels

    except FileNotFoundError:
        logging.error("TSV file not found.")
    except Exception as e:
        logging.error(f"Error occurred: {str(e)}")

    return None, None

In [3]:
def divide_dataset(df):
#     try:
#         train_data, test_data, train_labels, test_labels = train_test_split(images, labels, test_size=test_size, random_state=random_state)
#         train_data, val_data, train_labels, val_labels = train_test_split(train_data, train_labels, test_size=test_size, random_state=random_state)

#         logging.info("Dataset divided successfully.")
#         return train_data, train_labels, val_data, val_labels, test_data, test_labels

#     except ValueError as e:
#         logging.error(f"ValueError: {str(e)}")
#     except Exception as e:
#         logging.error(f"An error occurred: {str(e)}")

#     return None, None, None, None, None, None

    train_df, test_df = train_test_split(df, test_size=0.1)
    # we reset the indices to start from zero
    train_df.reset_index(drop=True, inplace=True)
    test_df.reset_index(drop=True, inplace=True)
    
    return train_df, test_df
     


In [4]:
import torch
from torch.utils.data import Dataset
from PIL import Image

class IAMDataset(Dataset):
    def __init__(self, root_dir, df, processor, max_target_length=128):
        self.root_dir = root_dir
        self.df = df
        self.processor = processor
        self.max_target_length = max_target_length

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        # get file name + text 
        file_name = self.df['segment_id'][idx]
        text = self.df['text_equiv'][idx]
        # prepare image (i.e. resize + normalize)
        image = Image.open(self.root_dir + file_name + ".png").convert("RGB")
        pixel_values = self.processor(image, return_tensors="pt").pixel_values
        # add labels (input_ids) by encoding the text
        labels = self.processor.tokenizer(text, 
                                          padding="max_length", 
                                          max_length=self.max_target_length).input_ids
        # important: make sure that PAD tokens are ignored by the loss function
        labels = [label if label != self.processor.tokenizer.pad_token_id else -100 for label in labels]

        encoding = {"pixel_values": pixel_values.squeeze(), "labels": torch.tensor(labels)}
        return encoding

In [5]:
data = load_data()

train_data, test_data= divide_dataset(data)

In [6]:
from transformers import TrOCRProcessor

processor = TrOCRProcessor.from_pretrained("trocr-processor")
train_dataset = IAMDataset(root_dir='dataset/Fraktur/',
                           df=train_data,
                           processor=processor)
eval_dataset = IAMDataset(root_dir='dataset/Fraktur/',
                           df=test_data,
                           processor=processor)

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


In [7]:
print("Number of training examples:", len(train_dataset))
print("Number of validation examples:", len(eval_dataset))

Number of training examples: 6827
Number of validation examples: 759


In [8]:
from transformers import VisionEncoderDecoderModel
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = VisionEncoderDecoderModel.from_pretrained("trocr-base-stage1")
model.to(device)

Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at trocr-base-stage1 and are newly initialized: ['encoder.pooler.dense.bias', 'encoder.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


VisionEncoderDecoderModel(
  (encoder): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=False)
              (key): Linear(in_features=768, out_features=768, bias=False)
              (value): Linear(in_features=768, out_features=768, bias=False)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_fea

In [9]:
# set special tokens used for creating the decoder_input_ids from the labels
model.config.decoder_start_token_id = processor.tokenizer.cls_token_id
model.config.pad_token_id = processor.tokenizer.pad_token_id
# make sure vocab size is set correctly
model.config.vocab_size = model.config.decoder.vocab_size

# set beam search parameters
model.config.eos_token_id = processor.tokenizer.sep_token_id
model.config.max_length = 64
model.config.early_stopping = True
model.config.no_repeat_ngram_size = 3
model.config.length_penalty = 2.0
model.config.num_beams = 4

In [10]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

training_args = Seq2SeqTrainingArguments(
    predict_with_generate=True,
    evaluation_strategy="steps",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    fp16=False, 
    output_dir="./",
    logging_steps=2,
    save_steps=1000,
    eval_steps=200,
)

In [ ]:
from datasets import load_metric

cer_metric = load_metric("cer")

def compute_cer(pred_ids, label_ids):
    pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id
    label_str = processor.batch_decode(label_ids, skip_special_tokens=True)

    cer = cer_metric.compute(predictions=pred_str, references=label_str)

    return cer

/tmp/ipykernel_37568/162233088.py:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  cer_metric = load_metric("cer")


In [ ]:
def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = processor.tokenizer.pad_token_id
    label_str = processor.batch_decode(labels_ids, skip_special_tokens=True)

    cer = cer_metric.compute(predictions=pred_str, references=label_str)

    return {"cer": cer}
     


In [ ]:
from transformers import default_data_collator

# instantiate trainer
trainer = Seq2SeqTrainer(
    model=model,
    tokenizer=processor.feature_extractor,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=default_data_collator,
)
trainer.train()